In [ ]:
# 전이 학습(Transfer Learning)

In [ ]:
# 1. [참고] 데이터 경로 및 압축 해제 상태 확인

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# 1. 데이터 다운로드 및 경로 설정
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=url, extract=True)
base_dir = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_extracted/cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

# 2. 데이터 전처리
train_gen = ImageDataGenerator(rescale=1./255.)
val_gen = ImageDataGenerator(rescale=1./255.)

train_data = train_gen.flow_from_directory(
  train_dir,
  target_size = (224, 224),
  batch_size = 32,
  class_mode = 'binary'
)
val_data = val_gen.flow_from_directory(
  val_dir,
  target_size = (224, 224),
  batch_size = 32,
  class_mode = 'binary'
)

# 3. 사전학습된 VGG16 모델 로딩
base_model = VGG16(
  weights='imagenet',
  include_top = False,
  input_shape = (224, 224, 3)
)

# 4. 모든 레이어 고정 (가중치 학습 방지)
base_model.trainable = False

# 5. 새로운 출력층(1) 추가(Fully Connected)
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# 6. 컴파일
model.compile(
  optimizer = Adam(learning_rate=1e-4),
  loss = 'binary_crossentropy',
  metrics = ['accuracy']
)

# 7. 학습
history = model.fit(
  train_data,
  epochs = 3,
  validation_data = val_data
)

# 8. summary
model.summary()

In [ ]:
# 2. 실습1

# VGG16 을 전이학습으로 사용하고, 데이터는 간단히 랜덤 이미지 데이터로 만들어서 구조 테스트

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
import numpy as np

# 1) 랜덤 이미지 데이터 생성
# 훈련 데이터 100개, 검증 데이터 20개
train_data = np.random.rand(100, 224, 224, 3)
train_labels = np.random.randint(0, 2, size=(100,1))
val_data = np.random.rand(20, 224, 224, 3)
val_labels = np.random.randint(0, 2, size=(20,1))

# 2) VGG16 사전 학습 모델 로딩
# 모든 레이어의 가중치를 고정 (Feature Extractor 로 사용)
# 입력 이미지에서 의미 있는 패턴(특징)을 자동 추출해주는 신경망의 앞쪽(Conv Layer) 부분
base_model = VGG16(
  weights='imagenet',
  include_top=False,
  # include_top=False: 마지막 분류층 임의 설정
  input_shape=(224, 224, 3)
)
for layer in base_model.layers:
  layer.trainable = False
  
# 3) 새로운 출력층 추가 (Fine-tunning Head)
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

# 4) 전체 모델 정의 및 컴파일
model = Model(inputs=base_model.input, outputs=output)
model.compile(
	optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy']
)

# 5) 모델 학습
history = model.fit(
  train_data, train_labels,
  epochs=3,
  batch_size=16,
  validation_data=(val_data, val_labels)
)

model.summary()

In [ ]:
# 3. 실습2

# 1) 일반 합성곱 신경망(CNN)으로 분류 예측

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D,BatchNormalization

# 랜덤 시드 고정
SEED = 12
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# CIFAR10 이미지 데이터셋
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 피처 스케일링
X_train = X_train / 255.
X_test = X_test / 255.

print(f"train data size: {X_train.shape} {y_train.shape}")
print(f"test data size: {X_test.shape} {y_test.shape}")

for i in range(1, 21):
  plt.subplot(4, 5, i)
  plt.imshow(X_train[i])
  plt.axis('off')
plt.tight_layout()
plt.show()

# 모델 구축
def build_cnn():
  model = Sequential()
  model.add(Input(shape=(32, 32, 3)))
  model.add(Conv2D(32, 5, strides=(2,2), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D())
  
  model.add(Conv2D(64, 5, strides=(2, 2), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D())
  
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(
		optimizer='adam',
		loss='sparse_categorical_crossentropy',
		metrics=['accuracy']
	)
  return model

cnn_model = build_cnn()
cnn_model.summary()

# 모델 학습
cnn_history = cnn_model.fit(
  X_train, y_train,
  batch_size = 256,
  epochs = 20,
  validation_split = 0.1,
  verbose = 1
)

# 20 epoch 까지 손실함수와 정확도를 그래프로 나타내는 함수
def plot_metrics(history):
  fig, axes = plt.subplots(1, 2, figsize=(10,5))
  
  # Loss: 손실함수
  axes[0].plot(range(1, 21), history.history['loss'][:20], label='Train')
  axes[0].plot(range(1, 21), history.history['val_loss'][:20], label='Validation')
  axes[0].set_title('Loss')
  axes[0].legend()
  
  # Accuracy: 예측 정확도
  axes[1].plot(range(1, 21), history.history['accuracy'][:20], label='Train')
  axes[1].plot(range(1, 21), history.history['val_accuracy'][:20], label='Validation')
  axes[1].set_title("Accuracy")
  axes[1].legend()
  
  plt.show()
  
plot_metrics(cnn_history)

In [ ]:
# 3. 실습2

# 2) 전이학습(Transfer Learning)으로 분류 예측

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
import matplotlib.pyplot as plt

# 데이터셋 생성 (cifar10)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train / 255.
X_test = X_test / 255.

# Pretrained 모델 가져오기 (ResNet50)
cnn_base = ResNet50(
  weights='imagenet',
  include_top=False,
  input_shape=(32, 32, 3)
)

# Transfer 모델 구축
def build_transfer():
  model = Sequential()
  model.add(cnn_base)
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  
  model.compile(
		optimizer='adam',
		loss='sparse_categorical_crossentropy',
		metrics=['accuracy']
	)
  
  return model

transfer_model = build_transfer()
transfer_model.summary()

# 모델 학습
tm_history = transfer_model.fit(
  X_train, y_train,
  epochs = 20,
  batch_size = 256,
  validation_split = 0.1,
  verbose = 1
)

# 20 epoch 까지 손실함수와 정확도를 그래프로 나타내는 함수
def plot_metrics(history):
  fig, axes = plt.subplots(1, 2, figsize=(10,5))
  
  axes[0].plot(range(1, 21), history.history['loss'][:20], label='Train')
  axes[0].plot(range(1, 21), history.history['val_loss'][:20], label='Validation')
  axes[0].set_title('Loss')
  axes[0].legend()
  
  axes[1].plot(range(1, 21), history.history['accuracy'][:20], label='Train')
  axes[1].plot(range(1, 21), history.history['val_accuracy'][:20], label='Validation')
  axes[1].set_title('Accuracy')
  axes[1].legend()
  
  plt.show()
plot_metrics(tm_history)